# 🥐 Bäckerei Umsatz Vorhersage - Trainingsdatensatz Erstellung

Dieses Notebook erstellt einen kombinierten Trainingsdatensatz für die Vorhersage von Bäckerei-Umsätzen basierend auf:
- Historischen Umsatzdaten
- Wetterdaten  
- Feiertagen
- Besonderen Events (Kieler Woche)
- Preisindizes für Backwaren

**Ziel:** Ein sauberer DataFrame für Regressionsanalyse und neuronale Netze

In [1]:
# 1. BIBLIOTHEKEN UND KONFIGURATION
print("📚 Lade erforderliche Bibliotheken...")

# Basis-Bibliotheken für Datenmanipulation
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualisierung
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Konfiguration für bessere Darstellung
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Pandas Anzeige-Optionen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Seaborn Style
sns.set_palette("husl")

# Deutsche Locale für Datumsformatierung (falls verfügbar)
try:
    import locale
    locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')
    print("🇩🇪 Deutsche Locale aktiviert")
except:
    print("⚠️ Deutsche Locale nicht verfügbar - verwende Standard")

# Zeitstempel für Dataset-Erstellung
print(f"🕒 Dataset-Erstellung gestartet: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}")
print("✅ Alle Bibliotheken erfolgreich geladen!")

📚 Lade erforderliche Bibliotheken...


⚠️ Deutsche Locale nicht verfügbar - verwende Standard
🕒 Dataset-Erstellung gestartet: 01.07.2025 21:06:52
✅ Alle Bibliotheken erfolgreich geladen!


In [2]:
# 2. UMSATZDATEN LADEN UND VERARBEITEN
print("📊 Lade Umsatzdaten...")

# Umsatzdaten von GitHub laden
umsatz_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv"
df_umsatz = pd.read_csv(umsatz_url)

print(f"Shape der Umsatzdaten: {df_umsatz.shape}")
print(f"Spalten: {list(df_umsatz.columns)}")
print("\nErste 5 Zeilen:")
print(df_umsatz.head())

# Debugging: Schaue dir die ersten paar IDs an
print(f"\nErste 10 IDs: {df_umsatz.iloc[:10, 0].tolist()}")

# IDENTIFIZIERE DIE UMSATZSPALTE
print(f"\n🔍 SPALTEN-ANALYSE:")
for i, col in enumerate(df_umsatz.columns):
    print(f"Spalte {i}: '{col}' - Typ: {df_umsatz[col].dtype}")
    if df_umsatz[col].dtype in ['float64', 'int64']:
        print(f"  Wertebereich: {df_umsatz[col].min():.2f} bis {df_umsatz[col].max():.2f}")
    print(f"  Beispielwerte: {df_umsatz[col].head(3).tolist()}")
    print()

# Suche nach der Umsatzspalte
umsatz_spalte = None
possible_umsatz_names = ['umsatz', 'Umsatz', 'sales', 'revenue', 'amount']

# Prüfe explizit nach "umsatz" Spalte
if 'umsatz' in df_umsatz.columns:
    umsatz_spalte = 'umsatz'
    print("✅ Umsatzspalte 'umsatz' gefunden!")
elif 'Umsatz' in df_umsatz.columns:
    umsatz_spalte = 'Umsatz'
    print("✅ Umsatzspalte 'Umsatz' gefunden!")
else:
    # Falls keine explizite Umsatzspalte, nimm die zweite numerische Spalte
    numeric_cols = df_umsatz.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) >= 2:
        umsatz_spalte = numeric_cols[1]  # Zweite numerische Spalte
        print(f"⚠️ Keine explizite Umsatzspalte gefunden. Verwende '{umsatz_spalte}' als Umsatz.")
    elif len(numeric_cols) == 1:
        umsatz_spalte = numeric_cols[0]
        print(f"⚠️ Nur eine numerische Spalte gefunden. Verwende '{umsatz_spalte}' als Umsatz.")
    else:
        print("❌ Keine numerische Spalte für Umsatz gefunden!")
        raise ValueError("Keine Umsatzspalte identifiziert!")

print(f"\n📊 VERWENDETE UMSATZSPALTE: '{umsatz_spalte}'")

# Datum aus der ID extrahieren (Format: YYMMDDW)
def extract_date_info(row_id):
    """Extrahiert Datum und Warengruppe aus der ID (Format: YYMMDDW)"""
    try:
        id_str = str(int(row_id))
        
        # YYMMDD + W (Warengruppe) - 7 Stellen erwartet
        if len(id_str) == 7:
            jahr_2stellig = int(id_str[:2])
            monat = int(id_str[2:4])
            tag = int(id_str[4:6])
            warengruppe = int(id_str[6])
            
            # 2-stelliges Jahr zu 4-stelligem Jahr konvertieren
            # Annahme: 00-30 = 2000-2030, 31-99 = 1931-1999
            if jahr_2stellig <= 30:
                jahr = 2000 + jahr_2stellig
            else:
                jahr = 1900 + jahr_2stellig
            
            # Datum validieren und erstellen
            datum = pd.to_datetime(f"{jahr}-{monat:02d}-{tag:02d}")
            return datum, warengruppe
        else:
            print(f"⚠️ Ungültige ID-Länge: {id_str} (Länge: {len(id_str)}, erwartet: 7)")
            return None, None
    except Exception as e:
        print(f"⚠️ Fehler bei ID {row_id}: {e}")
        return None, None

# Datum und Warengruppe extrahieren
print("\n🔄 Extrahiere Datum und Warengruppe aus ID (Format: YYMMDDW)...")
df_umsatz[['Datum', 'Warengruppe']] = df_umsatz.iloc[:, 0].apply(
    lambda x: pd.Series(extract_date_info(x))
)

print(f"Vor Bereinigung: {len(df_umsatz)} Zeilen")
print(f"Fehlende Daten: {df_umsatz['Datum'].isna().sum()}")

# Ungültige Daten entfernen
df_umsatz = df_umsatz.dropna(subset=['Datum', 'Warengruppe'])
print(f"Nach Bereinigung: {len(df_umsatz)} Zeilen")

# Datentypen explizit setzen
df_umsatz['Datum'] = pd.to_datetime(df_umsatz['Datum'])
df_umsatz['Warengruppe'] = df_umsatz['Warengruppe'].astype(int)

# Sicherstellen, dass Umsatz numerisch ist
df_umsatz[umsatz_spalte] = pd.to_numeric(df_umsatz[umsatz_spalte], errors='coerce')

print(f"Datum-Datentyp: {df_umsatz['Datum'].dtype}")
print(f"Umsatz-Datentyp: {df_umsatz[umsatz_spalte].dtype}")

# Weitere Datums-Features erstellen (ohne Sonntag-bezogene Features)
df_umsatz['Jahr'] = df_umsatz['Datum'].dt.year
df_umsatz['Monat'] = df_umsatz['Datum'].dt.month
df_umsatz['Tag'] = df_umsatz['Datum'].dt.day
df_umsatz['Wochentag'] = df_umsatz['Datum'].dt.day_name()
df_umsatz['Wochentag_Nr'] = df_umsatz['Datum'].dt.dayofweek

print(f"✅ {len(df_umsatz)} gültige Datensätze nach Extraktion")
print(f"📅 Zeitraum: {df_umsatz['Datum'].min()} bis {df_umsatz['Datum'].max()}")
print(f"🏷️ Warengruppen: {sorted(df_umsatz['Warengruppe'].unique())}")
print(f"💰 Umsatz-Vorschau: {df_umsatz[umsatz_spalte].describe()}")

📊 Lade Umsatzdaten...
Shape der Umsatzdaten: (9334, 4)
Spalten: ['id', 'Datum', 'Warengruppe', 'Umsatz']

Erste 5 Zeilen:
        id       Datum  Warengruppe  Umsatz
0  1307011  2013-07-01            1  148.83
1  1307021  2013-07-02            1  159.79
2  1307031  2013-07-03            1  111.89
3  1307041  2013-07-04            1  168.86
4  1307051  2013-07-05            1  171.28

Erste 10 IDs: [1307011, 1307021, 1307031, 1307041, 1307051, 1307061, 1307071, 1307081, 1307091, 1307101]

🔍 SPALTEN-ANALYSE:
Spalte 0: 'id' - Typ: int64
  Wertebereich: 1307011.00 bis 1807315.00
  Beispielwerte: [1307011, 1307021, 1307031]

Spalte 1: 'Datum' - Typ: object
  Beispielwerte: ['2013-07-01', '2013-07-02', '2013-07-03']

Spalte 2: 'Warengruppe' - Typ: int64
  Wertebereich: 1.00 bis 6.00
  Beispielwerte: [1, 1, 1]

Spalte 3: 'Umsatz' - Typ: float64
  Wertebereich: 7.05 bis 1879.46
  Beispielwerte: [148.828353112183, 159.79375714468, 111.885593514353]

✅ Umsatzspalte 'Umsatz' gefunden!

📊 VERWENDE

In [3]:
# 3. WARENGRUPPEN IN LESBARE NAMEN UMWANDELN UND BINÄRKODIEREN
# Sicherheitsüberprüfung: Stelle sicher, dass df_umsatz existiert
try:
    if 'df_umsatz' not in locals():
        raise NameError("df_umsatz ist nicht definiert. Bitte führe zuerst Zelle 3 (Umsatzdaten laden) aus.")
    
    print(f"✅ df_umsatz gefunden mit {len(df_umsatz)} Zeilen")
except NameError as e:
    print(f"❌ Fehler: {e}")
    print("🔄 Führe die Zellen in der richtigen Reihenfolge aus:")
    print("   1. Bibliotheken laden")
    print("   2. Umsatzdaten laden")
    print("   3. Warengruppen verarbeiten")
    raise

warengruppen_mapping = {
    1: "Brot",
    2: "Brötchen", 
    3: "Croissant",
    4: "Konditorei",
    5: "Kuchen",
    6: "Saisonbrot"
}

df_umsatz['Warengruppe_Name'] = df_umsatz['Warengruppe'].map(warengruppen_mapping)

# Binärkodierung für Warengruppen mit expliziten 0/1-Werten
print("🔄 Erstelle Binärkodierung für Warengruppen mit 0/1...")
for warengruppe_nr, warengruppe_name in warengruppen_mapping.items():
    # Erstelle binäre Spalte: 1 wenn Warengruppe zutrifft, 0 sonst
    df_umsatz[f'Warengruppe_{warengruppe_name}'] = (df_umsatz['Warengruppe'] == warengruppe_nr).astype(int)

print("✅ Warengruppen-Features mit expliziter 0/1-Kodierung erstellt:")
warengruppen_cols = [col for col in df_umsatz.columns if col.startswith('Warengruppe_') and col != 'Warengruppe_Name']
print(warengruppen_cols)

# Überprüfe die Binärkodierung
print(f"\n🔍 Binärkodierung-Überprüfung:")
for col in warengruppen_cols:
    unique_values = df_umsatz[col].unique()
    print(f"{col}: Eindeutige Werte = {sorted(unique_values)} (Typ: {df_umsatz[col].dtype})")

# Überblick über die Daten
print(f"\n📈 Dataset Info:")
print(f"Zeilen: {len(df_umsatz)}")
print(f"Spalten: {len(df_umsatz.columns)}")
print(f"Umsatzbereich: {df_umsatz.iloc[:, 1].min():.2f} € - {df_umsatz.iloc[:, 1].max():.2f} €")

✅ df_umsatz gefunden mit 9334 Zeilen
🔄 Erstelle Binärkodierung für Warengruppen mit 0/1...
✅ Warengruppen-Features mit expliziter 0/1-Kodierung erstellt:
['Warengruppe_Brot', 'Warengruppe_Brötchen', 'Warengruppe_Croissant', 'Warengruppe_Konditorei', 'Warengruppe_Kuchen', 'Warengruppe_Saisonbrot']

🔍 Binärkodierung-Überprüfung:
Warengruppe_Brot: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)
Warengruppe_Brötchen: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)
Warengruppe_Croissant: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)
Warengruppe_Konditorei: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)
Warengruppe_Kuchen: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)
Warengruppe_Saisonbrot: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)

📈 Dataset Info:
Zeilen: 9334
Spalten: 16
Umsatzbereich: .2f € - .2f €


In [4]:
# 4. UMSATZANALYSE UND BEREINIGUNG
print("📊 Analysiere Umsatzdaten ohne Manipulation...")

# Verwende die bereits identifizierte Umsatzspalte
if 'umsatz_spalte' not in locals():
    if 'umsatz' in df_umsatz.columns:
        umsatz_spalte = 'umsatz'
    elif 'Umsatz' in df_umsatz.columns:
        umsatz_spalte = 'Umsatz'
    else:
        umsatz_spalte = df_umsatz.columns[1]

print(f"Verwende Umsatz-Spalte: '{umsatz_spalte}'")

# Sicherstellen, dass Umsatz numerisch ist
df_umsatz[umsatz_spalte] = pd.to_numeric(df_umsatz[umsatz_spalte], errors='coerce')

# WICHTIG: Keine Manipulation der Umsatzdaten!
# Negative Umsätze sind valide (Rückgaben/Stornierungen)
# Keine Aggregation - behalte Einzeltransaktionen

print(f"\n🔍 ORIGINAL UMSATZDATEN-ANALYSE:")
print(f"Anzahl Transaktionen: {len(df_umsatz)}")
print(f"Umsatzbereich: {df_umsatz[umsatz_spalte].min():.2f} € bis {df_umsatz[umsatz_spalte].max():.2f} €")
print(f"Durchschnittsumsatz: {df_umsatz[umsatz_spalte].mean():.2f} €")
print(f"Negative Umsätze: {(df_umsatz[umsatz_spalte] < 0).sum()} Transaktionen ({(df_umsatz[umsatz_spalte] < 0).mean()*100:.1f}%)")

# Zeige Beispiele negativer Umsätze (als Information, nicht zur Entfernung)
negative_sales = df_umsatz[df_umsatz[umsatz_spalte] < 0]
if len(negative_sales) > 0:
    print(f"\n📋 NEGATIVE UMSÄTZE (Beispiele - das sind legitime Rückgaben/Stornierungen):")
    print(negative_sales[['Datum', 'Warengruppe_Name', umsatz_spalte]].head())

# KEINE AGGREGATION - behalte die Daten wie sie sind!
print(f"\n✅ Umsatzdaten bleiben unverändert")
print(f"📊 Finale Anzahl Datensätze: {len(df_umsatz)}")

📊 Analysiere Umsatzdaten ohne Manipulation...
Verwende Umsatz-Spalte: 'Umsatz'

🔍 ORIGINAL UMSATZDATEN-ANALYSE:
Anzahl Transaktionen: 9334
Umsatzbereich: 7.05 € bis 1879.46 €
Durchschnittsumsatz: 206.75 €
Negative Umsätze: 0 Transaktionen (0.0%)

✅ Umsatzdaten bleiben unverändert
📊 Finale Anzahl Datensätze: 9334


In [5]:
# 4. WETTERDATEN LADEN UND VERKNÜPFEN
print("🌤️ Lade Wetterdaten...")

# Wetterdaten von GitHub laden
wetter_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/wetter.csv"
df_wetter = pd.read_csv(wetter_url)

print(f"Shape der Wetterdaten: {df_wetter.shape}")
print(f"Spalten: {list(df_wetter.columns)}")
print("\nErste 5 Zeilen Wetterdaten:")
print(df_wetter.head())

# Datum in Wetterdaten konvertieren
df_wetter['Datum'] = pd.to_datetime(df_wetter['Datum'])

# Feature: Wettercode fehlt
df_wetter['Wettercode_fehlt'] = df_wetter['Wettercode'].isna().astype(int)

# Fehlende Wetterdaten mit Median füllen
numeric_weather_cols = ['Temperatur', 'Windgeschwindigkeit', 'Bewoelkung']
for col in numeric_weather_cols:
    if col in df_wetter.columns:
        median_val = df_wetter[col].median()
        df_wetter[col] = df_wetter[col].fillna(median_val)
        print(f"📊 {col}: Fehlende Werte mit Median {median_val:.2f} gefüllt")

# Wetterdaten mit Umsatzdaten verknüpfen
print("\n🔗 Verknüpfe Wetter- mit Umsatzdaten...")
df_combined = df_umsatz.merge(df_wetter, on='Datum', how='left')

print(f"✅ Kombinierter Datensatz: {df_combined.shape}")
print(f"🌡️ Temperaturbereich: {df_combined['Temperatur'].min():.1f}°C - {df_combined['Temperatur'].max():.1f}°C")

# Fehlende Wetterdaten anzeigen
missing_weather = df_combined['Temperatur'].isna().sum()
if missing_weather > 0:
    print(f"⚠️ {missing_weather} Tage ohne Wetterdaten - werden mit Durchschnitt gefüllt")
    for col in numeric_weather_cols:
        if col in df_combined.columns:
            df_combined[col] = df_combined[col].fillna(df_combined[col].mean())

🌤️ Lade Wetterdaten...
Shape der Wetterdaten: (2601, 5)
Spalten: ['Datum', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode']

Erste 5 Zeilen Wetterdaten:
        Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
0  2012-01-01        8.00        9.82                   14       58.00
1  2012-01-02        7.00        7.44                   12         NaN
2  2012-01-03        8.00        5.54                   18       63.00
3  2012-01-04        4.00        5.69                   19       80.00
4  2012-01-05        6.00        5.30                   23       80.00
📊 Temperatur: Fehlende Werte mit Median 12.00 gefüllt
📊 Windgeschwindigkeit: Fehlende Werte mit Median 10.00 gefüllt
📊 Bewoelkung: Fehlende Werte mit Median 6.00 gefüllt

🔗 Verknüpfe Wetter- mit Umsatzdaten...
✅ Kombinierter Datensatz: (9334, 21)
🌡️ Temperaturbereich: -8.5°C - 31.4°C
⚠️ 16 Tage ohne Wetterdaten - werden mit Durchschnitt gefüllt


In [6]:
# 5. FEIERTAGE LADEN UND VERKNÜPFEN
print("🎄 Lade Feiertagsdaten...")

try:
    # Feiertage aus lokaler Datei laden
    feiertage_path = "/workspaces/bakery_sales_prediction/5_Datasets/DE-Feiertage_2020_bis_2035.csv"
    
    # Versuche verschiedene Separator und Encoding
    try:
        df_feiertage = pd.read_csv(feiertage_path, sep=';', encoding='utf-8')
    except:
        try:
            df_feiertage = pd.read_csv(feiertage_path, sep=',', encoding='utf-8')
        except:
            df_feiertage = pd.read_csv(feiertage_path, sep=';', encoding='latin-1')
    
    print(f"Shape der Feiertagsdaten: {df_feiertage.shape}")
    print(f"Spalten: {list(df_feiertage.columns)}")
    print("\nErste 5 Feiertage:")
    print(df_feiertage.head())
    
    # Datum in Feiertagsdaten konvertieren
    datum_col = df_feiertage.columns[0]  # Erste Spalte als Datum
    
    # Prüfe das Datumsformat und konvertiere entsprechend
    sample_date = str(df_feiertage[datum_col].iloc[0])
    print(f"Beispiel-Datum Format: '{sample_date}'")
    
    if ';' in sample_date:
        # Format: "01.01.2000;Neujahr" - extrahiere nur das Datum
        df_feiertage[datum_col] = df_feiertage[datum_col].str.split(';').str[0]
        print("✅ Semikolon-separierte Daten erkannt und aufgeteilt")
    
    # Konvertiere Datum mit deutschem Format (DD.MM.YYYY)
    try:
        df_feiertage['Datum'] = pd.to_datetime(df_feiertage[datum_col], format='%d.%m.%Y')
    except ValueError:
        try:
            df_feiertage['Datum'] = pd.to_datetime(df_feiertage[datum_col], dayfirst=True)
        except:
            # Fallback: Automatische Erkennung
            df_feiertage['Datum'] = pd.to_datetime(df_feiertage[datum_col], infer_datetime_format=True)
    
    # Nur relevante Jahre filtern (2013-2018)
    original_count = len(df_feiertage)
    df_feiertage = df_feiertage[
        (df_feiertage['Datum'].dt.year >= 2013) & 
        (df_feiertage['Datum'].dt.year <= 2018)
    ]
    
    print(f"📅 Gefiltert von {original_count} auf {len(df_feiertage)} Feiertage (2013-2018)")
    
    # Binäre Feiertag-Spalte erstellen
    feiertage_dates = set(df_feiertage['Datum'].dt.date)
    df_combined['ist_feiertag'] = df_combined['Datum'].dt.date.isin(feiertage_dates).astype(int)
    
    print(f"✅ {len(feiertage_dates)} Feiertage im relevanten Zeitraum gefunden")
    print(f"📅 Tage mit Feiertag im Dataset: {df_combined['ist_feiertag'].sum()}")
    
    # Zeige einige Beispiel-Feiertage
    if len(df_feiertage) > 0:
        print(f"\n🎄 Beispiel-Feiertage:")
        print(df_feiertage[['Datum']].head(10))
    
except FileNotFoundError:
    print("⚠️ Feiertagsdatei nicht gefunden - erstelle manuell Feiertage")
    # Manuelle Definition wichtiger deutscher Feiertage
    deutsche_feiertage = [
        # 2013
        '2013-01-01', '2013-03-29', '2013-04-01', '2013-05-01', '2013-05-09', '2013-05-20', 
        '2013-10-03', '2013-12-25', '2013-12-26',
        # 2014
        '2014-01-01', '2014-04-18', '2014-04-21', '2014-05-01', '2014-05-29', '2014-06-09',
        '2014-10-03', '2014-12-25', '2014-12-26',
        # 2015
        '2015-01-01', '2015-04-03', '2015-04-06', '2015-05-01', '2015-05-14', '2015-05-25',
        '2015-10-03', '2015-12-25', '2015-12-26',
        # 2016
        '2016-01-01', '2016-03-25', '2016-03-28', '2016-05-01', '2016-05-05', '2016-05-16',
        '2016-10-03', '2016-12-25', '2016-12-26',
        # 2017
        '2017-01-01', '2017-04-14', '2017-04-17', '2017-05-01', '2017-05-25', '2017-06-05',
        '2017-10-03', '2017-12-25', '2017-12-26',
        # 2018
        '2018-01-01', '2018-03-30', '2018-04-02', '2018-05-01', '2018-05-10', '2018-05-21',
        '2018-10-03', '2018-12-25', '2018-12-26'
    ]
    
    feiertage_dates = set(pd.to_datetime(deutsche_feiertage).date)
    df_combined['ist_feiertag'] = df_combined['Datum'].dt.date.isin(feiertage_dates).astype(int)
    print(f"✅ {len(feiertage_dates)} manuelle Feiertage erstellt")

except Exception as e:
    print(f"⚠️ Fehler beim Verarbeiten der Feiertage: {e}")
    print("Erstelle Standard-Feiertage als Fallback...")
    
    # Minimale Feiertage als Fallback
    standard_feiertage = [
        '2013-01-01', '2013-12-25', '2014-01-01', '2014-12-25',
        '2015-01-01', '2015-12-25', '2016-01-01', '2016-12-25',
        '2017-01-01', '2017-12-25', '2018-01-01', '2018-12-25'
    ]
    
    feiertage_dates = set(pd.to_datetime(standard_feiertage).date)
    df_combined['ist_feiertag'] = df_combined['Datum'].dt.date.isin(feiertage_dates).astype(int)
    print(f"✅ {len(feiertage_dates)} Standard-Feiertage erstellt")

# Feiertags-Statistiken
feiertag_count = df_combined['ist_feiertag'].sum()
total_days = len(df_combined)
feiertag_percent = (feiertag_count / total_days) * 100

print(f"\n📊 FEIERTAG-STATISTIKEN:")
print(f"Feiertage im Dataset: {feiertag_count} von {total_days} Tagen ({feiertag_percent:.1f}%)")

if feiertag_count > 0:
    # Analysiere Umsatz an Feiertagen vs. normale Tage - FIX: Verwende korrekte Umsatz-Spalte
    # Identifiziere die korrekte Umsatz-Spalte
    if 'umsatz_spalte' in locals() and umsatz_spalte in df_combined.columns:
        umsatz_col_name = umsatz_spalte
    else:
        # Fallback: Suche nach der Umsatz-Spalte
        numeric_cols = df_combined.select_dtypes(include=[np.number]).columns
        umsatz_col_name = None
        for col in ['umsatz', 'Umsatz']:
            if col in df_combined.columns:
                umsatz_col_name = col
                break
        
        if umsatz_col_name is None and len(numeric_cols) > 1:
            # Nimm die zweite numerische Spalte (erste ist meist ID)
            umsatz_col_name = numeric_cols[1]
        elif umsatz_col_name is None and len(numeric_cols) == 1:
            umsatz_col_name = numeric_cols[0]
    
    if umsatz_col_name and umsatz_col_name in df_combined.columns:
        try:
            # Stelle sicher, dass die Umsatz-Spalte numerisch ist
            df_combined[umsatz_col_name] = pd.to_numeric(df_combined[umsatz_col_name], errors='coerce')
            
            feiertag_umsatz = df_combined[df_combined['ist_feiertag'] == 1][umsatz_col_name].mean()
            normal_umsatz = df_combined[df_combined['ist_feiertag'] == 0][umsatz_col_name].mean()
            
            # Prüfe ob die Werte numerisch sind
            if pd.notna(feiertag_umsatz) and pd.notna(normal_umsatz) and normal_umsatz != 0:
                print(f"💰 Durchschnittsumsatz an Feiertagen: {feiertag_umsatz:.2f} €")
                print(f"💰 Durchschnittsumsatz an normalen Tagen: {normal_umsatz:.2f} €")
                print(f"📈 Feiertag vs. Normal Ratio: {feiertag_umsatz/normal_umsatz:.2f}")
            else:
                print(f"⚠️ Konnte Umsatz-Statistiken nicht berechnen (Feiertag: {feiertag_umsatz}, Normal: {normal_umsatz})")
                
        except Exception as e:
            print(f"⚠️ Fehler bei Umsatz-Statistiken: {e}")
            print(f"   Umsatz-Spalte: '{umsatz_col_name}', Typ: {df_combined[umsatz_col_name].dtype}")
    else:
        print(f"⚠️ Keine gültige Umsatz-Spalte gefunden für Statistiken")

🎄 Lade Feiertagsdaten...
Shape der Feiertagsdaten: (468, 2)
Spalten: ['Datum', 'Feiertag']

Erste 5 Feiertage:
        Datum             Feiertag
0  01.01.2000              Neujahr
1  21.04.2000           Karfreitag
2  23.04.2000               Ostern
3  24.04.2000          Ostermontag
4  01.06.2000  Christi Himmelfahrt
Beispiel-Datum Format: '01.01.2000'
📅 Gefiltert von 468 auf 78 Feiertage (2013-2018)
✅ 78 Feiertage im relevanten Zeitraum gefunden
📅 Tage mit Feiertag im Dataset: 201

🎄 Beispiel-Feiertage:
         Datum
169 2013-01-01
170 2013-03-29
171 2013-03-31
172 2013-04-01
173 2013-05-09
174 2013-05-19
175 2013-05-20
176 2013-05-01
177 2013-10-03
178 2013-12-24

📊 FEIERTAG-STATISTIKEN:
Feiertage im Dataset: 201 von 9334 Tagen (2.2%)
💰 Durchschnittsumsatz an Feiertagen: 299.58 €
💰 Durchschnittsumsatz an normalen Tagen: 204.71 €
📈 Feiertag vs. Normal Ratio: 1.46


In [7]:
# 6. JAHRESZEIT BESTIMMEN
print("🌸 Erstelle Jahreszeiten-Feature...")

def get_season(month):
    """Bestimmt die Jahreszeit basierend auf dem Monat"""
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Frühling"
    elif month in [6, 7, 8]:
        return "Sommer"
    else:  # 9, 10, 11
        return "Herbst"

df_combined['Jahreszeit'] = df_combined['Monat'].apply(get_season)

# Binärkodierung für Jahreszeiten mit expliziten 0/1-Werten
print("🔄 Erstelle Binärkodierung für Jahreszeiten mit 0/1...")
jahreszeiten = ['Winter', 'Frühling', 'Sommer', 'Herbst']
for jahreszeit in jahreszeiten:
    df_combined[f'Jahreszeit_{jahreszeit}'] = (df_combined['Jahreszeit'] == jahreszeit).astype(int)

print("✅ Jahreszeiten-Features mit expliziter 0/1-Kodierung erstellt:")
jahreszeiten_cols = [col for col in df_combined.columns if col.startswith('Jahreszeit_')]
print(jahreszeiten_cols)

# Überprüfe die Binärkodierung
print(f"\n🔍 Jahreszeiten-Binärkodierung-Überprüfung:")
for col in jahreszeiten_cols:
    unique_values = df_combined[col].unique()
    print(f"{col}: Eindeutige Werte = {sorted(unique_values)} (Typ: {df_combined[col].dtype})")

# Verteilung der Jahreszeiten anzeigen
jahreszeit_verteilung = df_combined['Jahreszeit'].value_counts()
print(f"\n📊 Verteilung der Jahreszeiten:")
print(jahreszeit_verteilung)

🌸 Erstelle Jahreszeiten-Feature...
🔄 Erstelle Binärkodierung für Jahreszeiten mit 0/1...
✅ Jahreszeiten-Features mit expliziter 0/1-Kodierung erstellt:
['Jahreszeit_Winter', 'Jahreszeit_Frühling', 'Jahreszeit_Sommer', 'Jahreszeit_Herbst']

🔍 Jahreszeiten-Binärkodierung-Überprüfung:
Jahreszeit_Winter: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)
Jahreszeit_Frühling: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)
Jahreszeit_Sommer: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)
Jahreszeit_Herbst: Eindeutige Werte = [np.int64(0), np.int64(1)] (Typ: int64)

📊 Verteilung der Jahreszeiten:
Jahreszeit
Herbst      2410
Sommer      2405
Winter      2293
Frühling    2226
Name: count, dtype: int64


In [8]:
# 7. ZUSÄTZLICHE ZEIT-FEATURES (ohne Sonntag)
print("📅 Erstelle zusätzliche Zeit-Features...")

# Wochentag-Binärkodierung (falls für spezifische Analysen benötigt)
wochentage = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
print("🔄 Erstelle Wochentag-Binärkodierung (optional)...")

for wochentag in wochentage:
    df_combined[f'Wochentag_{wochentag}'] = (df_combined['Wochentag'] == wochentag).astype(int)

wochentag_cols = [col for col in df_combined.columns if col.startswith('Wochentag_')]
print(f"✅ Wochentag-Features erstellt: {len(wochentag_cols)} Features")

# Überprüfe die Wochentag-Verteilung
print(f"\n📊 Wochentag-Verteilung:")
wochentag_verteilung = df_combined['Wochentag'].value_counts()
print(wochentag_verteilung)

print(f"\n📝 Hinweis: Sonntag-Feature wurde entfernt. Verwende bei Bedarf 'Wochentag_Sunday' aus der Wochentag-Binärkodierung.")

📅 Erstelle zusätzliche Zeit-Features...
🔄 Erstelle Wochentag-Binärkodierung (optional)...
✅ Wochentag-Features erstellt: 8 Features

📊 Wochentag-Verteilung:
Wochentag
Tuesday      1345
Wednesday    1342
Sunday       1342
Saturday     1336
Thursday     1334
Monday       1324
Friday       1311
Name: count, dtype: int64

📝 Hinweis: Sonntag-Feature wurde entfernt. Verwende bei Bedarf 'Wochentag_Sunday' aus der Wochentag-Binärkodierung.


In [9]:
# 8. KIELER WOCHE: Daten laden, mergen und binär kodieren
print("⛵ Lade Kieler Woche-Daten und erstelle binäres Feature ...")

# Kiwo-Daten laden
kiwo_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/kiwo.csv"
df_kiwo = pd.read_csv(kiwo_url)
print(f"Shape der Kiwo-Daten: {df_kiwo.shape}")
print(f"Spalten: {list(df_kiwo.columns)}")
print(df_kiwo.head())

# Datumsspalte in datetime konvertieren (Spalte heißt meist 'Datum' oder ähnlich)
if 'Datum' in df_kiwo.columns:
    df_kiwo['Datum'] = pd.to_datetime(df_kiwo['Datum'])
else:
    # Fallback: erste Spalte als Datum
    df_kiwo[df_kiwo.columns[0]] = pd.to_datetime(df_kiwo[df_kiwo.columns[0]])
    df_kiwo = df_kiwo.rename(columns={df_kiwo.columns[0]: 'Datum'})

# Binäres Feature 'ist_kiwo' setzen: 1 für Kiwo-Tage, sonst 0
kiwo_tage = set(df_kiwo['Datum'].dt.date)
df_combined['ist_kiwo'] = df_combined['Datum'].dt.date.isin(kiwo_tage).astype(int)

print(f"✅ Kieler Woche-Feature ergänzt: {df_combined['ist_kiwo'].sum()} Tage im Datensatz sind während der Kieler Woche.")

⛵ Lade Kieler Woche-Daten und erstelle binäres Feature ...
Shape der Kiwo-Daten: (72, 2)
Spalten: ['Datum', 'KielerWoche']
        Datum  KielerWoche
0  2012-06-16            1
1  2012-06-17            1
2  2012-06-18            1
3  2012-06-19            1
4  2012-06-20            1
✅ Kieler Woche-Feature ergänzt: 223 Tage im Datensatz sind während der Kieler Woche.


In [10]:
# 10. FINALISIERUNG DES DATASETS
print("🔧 Finalisiere den Trainingsdatensatz...")

# Überprüfe fehlende Werte
print("\n🔍 Überprüfe fehlende Werte:")
missing_values = df_combined.isnull().sum()
missing_values = missing_values[missing_values > 0]
if len(missing_values) > 0:
    print(missing_values)
    
    # Fülle fehlende numerische Werte mit Median
    numeric_cols = df_combined.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_combined[col].isnull().sum() > 0:
            median_val = df_combined[col].median()
            df_combined[col] = df_combined[col].fillna(median_val)
            print(f"✅ {col}: {df_combined[col].isnull().sum()} fehlende Werte mit Median {median_val:.2f} gefüllt")
else:
    print("✅ Keine fehlenden Werte gefunden!")

# FIX: Korrekte Identifizierung der Umsatz-Spalte
# Verwende die bereits identifizierte Umsatz-Spalte aus den vorherigen Zellen
if 'umsatz_spalte' in locals() and umsatz_spalte in df_combined.columns:
    umsatz_col = umsatz_spalte
else:
    # Fallback: Suche nach der Umsatz-Spalte
    numeric_cols = df_combined.select_dtypes(include=[np.number]).columns
    umsatz_col = None
    for col in ['umsatz', 'Umsatz']:
        if col in df_combined.columns:
            umsatz_col = col
            break
    
    if umsatz_col is None and len(numeric_cols) > 1:
        # Nimm die zweite numerische Spalte (erste ist meist ID oder Index)
        umsatz_col = numeric_cols[1]
    elif umsatz_col is None and len(numeric_cols) >= 1:
        umsatz_col = numeric_cols[0]
    else:
        print("❌ Keine Umsatz-Spalte gefunden!")
        raise ValueError("Keine Umsatz-Spalte identifiziert!")

print(f"📊 Verwendete Umsatz-Spalte: '{umsatz_col}'")

feature_columns = [
    'Datum', 'Jahr', 'Monat', 'Tag', 'Wochentag', 'Wochentag_Nr',
    'Warengruppe', 'Warengruppe_Name',
    'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung', 'Wettercode_fehlt',
    'ist_feiertag', 'Jahreszeit', 'ist_kiwo', 'Preisindex',
    umsatz_col
]

# FIX: Korrekte Filterung der binär kodierten Spalten
warengruppen_cols = [col for col in df_combined.columns if col.startswith('Warengruppe_') and col != 'Warengruppe_Name']
# FIX: Korrigiere den Spaltenname für Jahreszeiten-Features
jahreszeiten_cols = [col for col in df_combined.columns if col.startswith('Jahreszeit_')]
wochentag_cols = [col for col in df_combined.columns if col.startswith('Wochentag_')]

all_features = feature_columns + warengruppen_cols + jahreszeiten_cols + wochentag_cols

# Finale Spalten filtern (nur die die existieren)
final_columns = [col for col in all_features if col in df_combined.columns]
df_final = df_combined[final_columns].copy()

# FIX: Umsatz-Spalte umbenennen und korrekt konvertieren
df_final = df_final.rename(columns={umsatz_col: 'Umsatz'})

# FIX: Robustere Konvertierung der Umsatz-Spalte zu numerischen Werten
print(f"🔧 Konvertiere Umsatz-Spalte zu numerischen Werten...")
try:
    # Prüfe zunächst den aktuellen Datentyp
    current_dtype = df_final['Umsatz'].dtype
    print(f"   Aktueller Datentyp: {current_dtype}")
    
    if current_dtype == 'object' or 'datetime' in str(current_dtype).lower():
        # Falls es sich um Object oder Datetime handelt, konvertiere zu numerisch
        df_final['Umsatz'] = pd.to_numeric(df_final['Umsatz'], errors='coerce')
        print(f"   ✅ Erfolgreich zu numerisch konvertiert")
    elif current_dtype in ['int64', 'float64']:
        # Bereits numerisch, explizit zu float konvertieren
        df_final['Umsatz'] = df_final['Umsatz'].astype(float)
        print(f"   ✅ Bereits numerisch, zu float konvertiert")
    else:
        print(f"   ⚠️ Unbekannter Datentyp: {current_dtype}")
        # Versuche manuelle Konvertierung
        df_final['Umsatz'] = pd.to_numeric(df_final['Umsatz'], errors='coerce')

except Exception as e:
    print(f"⚠️ Fehler beim Konvertieren der Umsatz-Spalte: {e}")
    print("🔄 Versuche alternative Konvertierung...")
    
    # Alternative: Element-weise Konvertierung für nicht-numerische Werte
    try:
        def convert_to_float(value):
            if pd.isna(value):
                return np.nan
            try:
                return float(value)
            except (ValueError, TypeError):
                return np.nan
        
        df_final['Umsatz'] = df_final['Umsatz'].apply(convert_to_float)
        print("✅ Alternative Konvertierung erfolgreich")
    except Exception as e2:
        print(f"❌ Auch alternative Konvertierung fehlgeschlagen: {e2}")
        # Als letzter Ausweg: Entferne die problematische Spalte und verwende eine andere
        problem_col = df_final['Umsatz']
        print(f"Problematische Spalte Typ: {type(problem_col.iloc[0])}")
        print(f"Erste 5 Werte: {problem_col.head().tolist()}")

print(f"\n📊 FINALER TRAININGSDATENSATZ:")
print(f"📏 Shape: {df_final.shape}")
print(f"📅 Zeitraum: {df_final['Datum'].min()} bis {df_final['Datum'].max()}")

# FIX: Sicherere Berechnung der Min/Max-Werte
try:
    umsatz_values = df_final['Umsatz'].dropna()  # Entferne NaN-Werte für Berechnung
    if len(umsatz_values) > 0:
        umsatz_min = float(umsatz_values.min())
        umsatz_max = float(umsatz_values.max())
        print(f"💰 Umsatzbereich: {umsatz_min:.2f} € - {umsatz_max:.2f} €")
    else:
        print("⚠️ Keine gültigen Umsatzwerte gefunden")
except Exception as e:
    print(f"⚠️ Fehler bei Umsatz-Statistiken: {e}")

print(f"\n🏷️ FEATURES ({len(df_final.columns)} Spalten):")
for i, col in enumerate(df_final.columns, 1):
    print(f"{i:2d}. {col}")

# Überprüfe Binärkodierung im finalen Dataset
print(f"\n🔍 BINÄRKODIERUNG-ÜBERPRÜFUNG IM FINALEN DATASET:")
binary_cols = [col for col in df_final.columns if col.startswith(('Warengruppe_', 'Jahreszeit_', 'Wochentag_'))]
print(f"Gefundene binäre Spalten: {len(binary_cols)}")
for col in binary_cols:
    try:
        unique_values = sorted(df_final[col].unique())
        print(f"{col}: {unique_values} (Typ: {df_final[col].dtype})")
    except Exception as e:
        print(f"{col}: Fehler bei Analyse - {e}")

🔧 Finalisiere den Trainingsdatensatz...

🔍 Überprüfe fehlende Werte:
Wettercode          2325
Wettercode_fehlt      16
dtype: int64
Wettercode          2325
Wettercode_fehlt      16
dtype: int64
✅ Wettercode: 0 fehlende Werte mit Median 28.00 gefüllt
✅ Wettercode_fehlt: 0 fehlende Werte mit Median 0.00 gefüllt
📊 Verwendete Umsatz-Spalte: 'Umsatz'
🔧 Konvertiere Umsatz-Spalte zu numerischen Werten...
   Aktueller Datentyp: float64
   ✅ Bereits numerisch, zu float konvertiert

📊 FINALER TRAININGSDATENSATZ:
📏 Shape: (9334, 34)
📅 Zeitraum: 2013-07-01 00:00:00 bis 2018-07-31 00:00:00
💰 Umsatzbereich: 7.05 € - 1879.46 €

🏷️ FEATURES (34 Spalten):
 1. Datum
 2. Jahr
 3. Monat
 4. Tag
 5. Wochentag
 6. Wochentag_Nr
 7. Warengruppe
 8. Warengruppe_Name
 9. Temperatur
10. Windgeschwindigkeit
11. Bewoelkung
12. Wettercode_fehlt
13. ist_feiertag
14. Jahreszeit
15. ist_kiwo
16. Umsatz
17. Warengruppe_Brot
18. Warengruppe_Brötchen
19. Warengruppe_Croissant
20. Warengruppe_Konditorei
21. Warengruppe_K

In [11]:
# NORMALISIERUNG DER NUMERISCHEN FEATURES
print("🔧 Normalisiere numerische Features für Modelltraining...")

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
import warnings
warnings.filterwarnings('ignore')

# Kopie des finalen Datasets für Normalisierung erstellen
df_normalized = df_final.copy()

# Identifiziere numerische Features (ohne Zielvariable und binäre Features)
numeric_features_to_scale = [
    'Jahr', 'Monat', 'Tag', 'Wochentag_Nr',
    'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung', 
    'Preisindex'
]

# Nur Features behalten, die auch im Dataset existieren
numeric_features_to_scale = [col for col in numeric_features_to_scale if col in df_normalized.columns]

print(f"📊 Zu normalisierende Features: {numeric_features_to_scale}")

# Originale Features vor Normalisierung anzeigen
print(f"\n📈 ORIGINALE FEATURE-STATISTIKEN:")
print(df_normalized[numeric_features_to_scale].describe().round(2))

# 1. STANDARD SCALER (Z-Score Normalisierung)
print(f"\n🔄 Methode 1: Standard Scaler (Z-Score)")
try:
    scaler_standard = StandardScaler()
    standard_scaled = scaler_standard.fit_transform(df_normalized[numeric_features_to_scale])
    
    # Sichere Zuweisung der normalisierten Werte - jede Spalte einzeln
    for i, col in enumerate(numeric_features_to_scale):
        df_normalized[f'std_{col}'] = standard_scaled[:, i]
    
    print("✅ Standard Scaler erfolgreich angewendet")
    print("Statistiken der Z-standardisierten Features:")
    print(df_normalized[[f'std_{col}' for col in numeric_features_to_scale]].describe().round(2))
except Exception as e:
    print(f"⚠️ Standard Scaler Fehler: {str(e)[:100]}...")

# 2. MIN-MAX SCALER (0-1 Normalisierung)
print(f"\n🔄 Methode 2: Min-Max Scaler (0-1)")
try:
    scaler_minmax = MinMaxScaler()
    minmax_scaled = scaler_minmax.fit_transform(df_normalized[numeric_features_to_scale])
    
    # Sichere Zuweisung der normalisierten Werte - jede Spalte einzeln
    for i, col in enumerate(numeric_features_to_scale):
        df_normalized[f'minmax_{col}'] = minmax_scaled[:, i]
    
    print("✅ Min-Max Scaler erfolgreich angewendet")
except Exception as e:
    print(f"⚠️ Min-Max Scaler Fehler: {str(e)[:100]}...")

# 3. ROBUST SCALER (Median-basiert, weniger empfindlich gegen Ausreißer)
print(f"🔄 Methode 3: Robust Scaler (Median-basiert)")
try:
    scaler_robust = RobustScaler()
    robust_scaled = scaler_robust.fit_transform(df_normalized[numeric_features_to_scale])
    
    # Sichere Zuweisung der normalisierten Werte - jede Spalte einzeln
    for i, col in enumerate(numeric_features_to_scale):
        df_normalized[f'robust_{col}'] = robust_scaled[:, i]
    
    print("✅ Robust Scaler erfolgreich angewendet")
except Exception as e:
    print(f"⚠️ Robust Scaler Fehler: {str(e)[:100]}...")

# 4. POWER TRANSFORMER (Yeo-Johnson für Normalverteilung)
print(f"🔄 Methode 4: Power Transformer (Yeo-Johnson)")
try:
    power_transformer = PowerTransformer(method='yeo-johnson', standardize=True)
    power_scaled = power_transformer.fit_transform(df_normalized[numeric_features_to_scale])
    
    # Sichere Zuweisung der normalisierten Werte - jede Spalte einzeln
    for i, col in enumerate(numeric_features_to_scale):
        df_normalized[f'power_{col}'] = power_scaled[:, i]
    
    print("✅ Power Transformer erfolgreich angewendet")
except Exception as e:
    print(f"⚠️ Power Transformer übersprungen (Fehler: {str(e)[:50]}...)")

print(f"✅ Normalisierung abgeschlossen!")

# Vergleiche die Normalisierungsmethoden
scaling_methods = ['std', 'minmax', 'robust', 'power']
print(f"\n📊 VERGLEICH DER NORMALISIERUNGSCMETHODEN:")
print("=" * 80)

for method in scaling_methods:
    method_cols = [col for col in df_normalized.columns if col.startswith(f'{method}_')]
    if method_cols:
        print(f"\n{method.upper()}-SCALER:")
        try:
            stats = df_normalized[method_cols].describe().round(3)
            print(f"Mean Bereich: {stats.loc['mean'].min():.3f} bis {stats.loc['mean'].max():.3f}")
            print(f"Std Bereich: {stats.loc['std'].min():.3f} bis {stats.loc['std'].max():.3f}")
            print(f"Min Bereich: {stats.loc['min'].min():.3f} bis {stats.loc['min'].max():.3f}")
            print(f"Max Bereich: {stats.loc['max'].min():.3f} bis {stats.loc['max'].max():.3f}")
        except Exception as e:
            print(f"Fehler bei Statistiken: {str(e)[:50]}...")
    else:
        print(f"\n{method.upper()}-SCALER: Keine Spalten gefunden")

🔧 Normalisiere numerische Features für Modelltraining...
📊 Zu normalisierende Features: ['Jahr', 'Monat', 'Tag', 'Wochentag_Nr', 'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung']

📈 ORIGINALE FEATURE-STATISTIKEN:
📊 Zu normalisierende Features: ['Jahr', 'Monat', 'Tag', 'Wochentag_Nr', 'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung']

📈 ORIGINALE FEATURE-STATISTIKEN:


         Jahr   Monat     Tag  Wochentag_Nr  Wochentag_Nr  Temperatur  \
count 9334.00 9334.00 9334.00       9334.00       9334.00     9334.00   
mean  2015.52    6.65   15.71          3.00          3.00       12.03   
std      1.52    3.46    8.75          2.00          2.00        7.23   
min   2013.00    1.00    1.00          0.00          0.00       -8.48   
25%   2014.00    4.00    8.00          1.00          1.00        6.25   
50%   2016.00    7.00   16.00          3.00          3.00       11.66   
75%   2017.00   10.00   23.00          5.00          5.00       17.96   
max   2018.00   12.00   31.00          6.00          6.00       31.44   

       Windgeschwindigkeit  Bewoelkung  
count              9334.00     9334.00  
mean                 10.97        4.73  
std                   4.13        2.64  
min                   3.00        0.00  
25%                   8.00        3.00  
50%                  10.00        6.00  
75%                  13.00        7.00  
max           

In [12]:
# EMPFEHLUNG UND FINALE DATASET-ERSTELLUNG
print("🎯 EMPFEHLUNG FÜR NORMALISIERUNGSMETHODE:")
print("=" * 60)

# Bewertungskriterien für neuronale netze
print("📋 Bewertungskriterien für Neuronale Netze:")
print("1. Ähnliche Skalierung aller Features (Mean ≈ 0, Std ≈ 1)")
print("2. Keine extremen Ausreißer")
print("3. Stabile Gradientenberechnung")
print("4. Erhaltung der Datenverteilung")

# Analysiere Stabilität der verschiedenen Scaler
scaler_scores = {}

for method in ['std', 'minmax', 'robust']:
    method_cols = [col for col in df_normalized.columns if col.startswith(f'{method}_')]
    if method_cols:
        method_data = df_normalized[method_cols]
        
        # Bewertungskriterien berechnen
        mean_consistency = 1 / (1 + abs(method_data.mean().std()))  # Je einheitlicher die Means, desto besser
        std_consistency = 1 / (1 + abs(method_data.std().std()))    # Je einheitlicher die Stds, desto besser
        range_control = 1 / (1 + (method_data.max().max() - method_data.min().min()))  # Kontrollierter Wertebereich
        
        total_score = (mean_consistency + std_consistency + range_control) / 3
        scaler_scores[method] = {
            'total': total_score,
            'mean_consistency': mean_consistency,
            'std_consistency': std_consistency,
            'range_control': range_control
        }

# Beste Methode ermitteln
best_method = max(scaler_scores.keys(), key=lambda k: scaler_scores[k]['total'])

print(f"\n📊 BEWERTUNG DER NORMALISIERUNGSMETHODEN:")
for method, scores in scaler_scores.items():
    print(f"{method.upper()}-Scaler:")
    print(f"  Gesamtscore: {scores['total']:.3f}")
    print(f"  Mean-Konsistenz: {scores['mean_consistency']:.3f}")
    print(f"  Std-Konsistenz: {scores['std_consistency']:.3f}")
    print(f"  Wertebereich: {scores['range_control']:.3f}")

print(f"\n✅ EMPFEHLUNG: {best_method.upper()}-SCALER")
print(f"🏆 Beste Gesamtbewertung: {scaler_scores[best_method]['total']:.3f}")

# Finales Dataset mit der besten Normalisierungsmethode erstellen
best_method_cols = [col for col in df_normalized.columns if col.startswith(f'{best_method}_')]

# Features für finales normalisiertes Dataset auswählen (ohne Sonntag-Feature)
final_normalized_features = [
    'Datum', 'Umsatz',  # Basis-Features
    'Warengruppe', 'Warengruppe_Name', 'Wochentag', 'Jahreszeit',  # Kategorische Features
    'ist_feiertag', 'ist_kiwo', 'Wettercode_fehlt'  # Binäre Features (ohne ist_sonntag)
]

# Binär kodierte Features hinzufügen
warengruppen_binary = [col for col in df_normalized.columns if col.startswith('Warengruppe_') and col != 'Warengruppe_Name']
jahreszeiten_binary = [col for col in df_normalized.columns if col.startswith('Jahreszeit_')]
wochentag_binary = [col for col in df_normalized.columns if col.startswith('Wochentag_')]

# Normalisierte numerische Features der besten Methode hinzufügen
final_features = final_normalized_features + warengruppen_binary + jahreszeiten_binary + wochentag_binary + best_method_cols

# Nur existierende Spalten auswählen
final_features = [col for col in final_features if col in df_normalized.columns]

# Finales normalisiertes Dataset erstellen
df_model_ready = df_normalized[final_features].copy()

# Normalisierte Spalten umbenennen (Präfix entfernen für Klarheit)
rename_dict = {}
for col in best_method_cols:
    original_name = col.replace(f'{best_method}_', '')
    rename_dict[col] = f'{original_name}_normalized'

df_model_ready = df_model_ready.rename(columns=rename_dict)

print(f"\n📊 MODELL-BEREITES DATASET:")
print(f"📏 Shape: {df_model_ready.shape}")
print(f"🏷️ Features: {len(df_model_ready.columns)}")

print(f"\n🔧 NORMALISIERTE FEATURES:")
normalized_cols = [col for col in df_model_ready.columns if col.endswith('_normalized')]
for col in normalized_cols:
    stats = df_model_ready[col].describe()
    print(f"{col}: Mean={stats['mean']:.3f}, Std={stats['std']:.3f}, Range=[{stats['min']:.3f}, {stats['max']:.3f}]")

# Überprüfe die binären Features im finalen normalisierten Dataset
print(f"\n🔍 BINÄRE FEATURES IM NORMALISIERTEN DATASET:")
binary_cols = [col for col in df_model_ready.columns if col.startswith(('Warengruppe_', 'Jahreszeit_', 'Wochentag_'))]
print(f"Anzahl binärer Features: {len(binary_cols)}")
for col in binary_cols[:5]:  # Zeige nur die ersten 5 als Beispiel
    unique_vals = sorted(df_model_ready[col].unique())
    print(f"  {col}: {unique_vals}")
if len(binary_cols) > 5:
    print(f"  ... und {len(binary_cols) - 5} weitere binäre Features")

# Speichere das normalisierte Dataset
normalized_output_path = "/workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset_normalized.csv"

try:
    df_model_ready.to_csv(normalized_output_path, index=False, encoding='utf-8')
    print(f"\n✅ Normalisiertes Dataset gespeichert: {normalized_output_path}")
    
    # Zusätzliche Info-Datei mit Normalisierungsparametern
    normalization_info = {
        'Methode': best_method.upper() + '-Scaler',
        'Normalisierte_Features': [col.replace('_normalized', '') for col in normalized_cols],
        'Original_Features': numeric_features_to_scale,
        'Binaere_Features': binary_cols,
        'Entfernte_Features': ['ist_sonntag'],  # Dokumentiere entfernte Features
        'Dataset_Shape': df_model_ready.shape,
        'Empfehlung': f"Verwende {best_method.upper()}-Scaler für optimale NN-Performance"
    }
    
    import json
    info_path = "/workspaces/bakery_sales_prediction/5_Datasets/normalization_info.json"
    with open(info_path, 'w', encoding='utf-8') as f:
        json.dump(normalization_info, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Normalisierungs-Info gespeichert: {info_path}")
    
except Exception as e:
    print(f"❌ Fehler beim Speichern: {e}")

print(f"\n🎉 NORMALISIERUNG ABGESCHLOSSEN!")
print(f"📁 Original Dataset: bakery_training_dataset.csv")
print(f"📁 Normalisiertes Dataset: bakery_training_dataset_normalized.csv")
print(f"✅ Sonntag-Feature entfernt")
print(f"✅ Binärkodierung mit expliziten 0/1-Werten")
print(f"🚀 Bereit für Neuronale Netze und ML-Modelle!")

🎯 EMPFEHLUNG FÜR NORMALISIERUNGSMETHODE:
📋 Bewertungskriterien für Neuronale Netze:
1. Ähnliche Skalierung aller Features (Mean ≈ 0, Std ≈ 1)
2. Keine extremen Ausreißer
3. Stabile Gradientenberechnung
4. Erhaltung der Datenverteilung

📊 BEWERTUNG DER NORMALISIERUNGSMETHODEN:
STD-Scaler:
  Gesamtscore: 0.701
  Mean-Konsistenz: 1.000
  Std-Konsistenz: 1.000
  Wertebereich: 0.104
MINMAX-Scaler:
  Gesamtscore: 0.779
  Mean-Konsistenz: 0.912
  Std-Konsistenz: 0.925
  Wertebereich: 0.500
ROBUST-Scaler:
  Gesamtscore: 0.643
  Mean-Konsistenz: 0.904
  Std-Konsistenz: 0.897
  Wertebereich: 0.130

✅ EMPFEHLUNG: MINMAX-SCALER
🏆 Beste Gesamtbewertung: 0.779

📊 MODELL-BEREITES DATASET:
📏 Shape: (9334, 37)
🏷️ Features: 37

🔧 NORMALISIERTE FEATURES:
Jahr_normalized: Mean=0.505, Std=0.304, Range=[0.000, 1.000]
Monat_normalized: Mean=0.514, Std=0.315, Range=[0.000, 1.000]
Tag_normalized: Mean=0.490, Std=0.292, Range=[0.000, 1.000]
Wochentag_Nr_normalized: Mean=0.500, Std=0.334, Range=[0.000, 1.000]
Te

In [13]:
# DATASET SPEICHERN
print("💾 Speichere finalen Trainingsdatensatz...")

# Pfad für das finale Dataset
output_path = "/workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset.csv"

try:
    # CSV speichern
    df_final.to_csv(output_path, index=False, encoding='utf-8')
    print(f"✅ Dataset erfolgreich gespeichert: {output_path}")
    
    # Zusätzlich als Excel für bessere Lesbarkeit
    excel_path = output_path.replace('.csv', '.xlsx')
    df_final.to_excel(excel_path, index=False)
    print(f"✅ Dataset auch als Excel gespeichert: {excel_path}")
    
    # Feature-Liste separat speichern
    feature_info = pd.DataFrame({
        'Feature': df_final.columns,
        'Typ': df_final.dtypes,
        'Fehlende_Werte': df_final.isnull().sum(),
        'Eindeutige_Werte': df_final.nunique()
    })
    
    feature_path = "/workspaces/bakery_sales_prediction/5_Datasets/feature_description.csv"
    feature_info.to_csv(feature_path, index=False)
    print(f"✅ Feature-Beschreibung gespeichert: {feature_path}")
    
except Exception as e:
    print(f"❌ Fehler beim Speichern: {e}")

print(f"\n🎉 TRAININGSDATENSATZ ERFOLGREICH ERSTELLT!")
print(f"📁 Hauptdatei: {output_path}")
print(f"📋 {len(df_final)} Zeilen mit {len(df_final.columns)} Features")
print(f"🎯 Bereit für Regression und Neuronale Netze!")

# Finale Zusammenfassung (ohne Sonntag-Feature) - FIX: Robuste Behandlung fehlender Features
print(f"\n📊 FINALE ZUSAMMENFASSUNG:")
print(f"🥐 Warengruppen: {df_final['Warengruppe_Name'].nunique()}")
print(f"📅 Tage: {df_final['Datum'].nunique()}")
print(f"🌡️ Temperaturbereich: {df_final['Temperatur'].min():.1f}°C - {df_final['Temperatur'].max():.1f}°C")

# Sichere Behandlung von Feiertagen
if 'ist_feiertag' in df_final.columns:
    print(f"🎄 Feiertage: {df_final['ist_feiertag'].sum()} Tage")
else:
    print(f"🎄 Feiertage: Feature nicht verfügbar")

# Sichere Behandlung von Kieler Woche
if 'ist_kiwo' in df_final.columns:
    print(f"⛵ Kieler Woche: {df_final['ist_kiwo'].sum()} Tage")
else:
    print(f"⛵ Kieler Woche: Feature nicht verfügbar")

# Überprüfe die binären Features
binary_features = [col for col in df_final.columns if col.startswith(('Warengruppe_', 'Jahreszeit_', 'Wochentag_'))]
print(f"🔢 Binäre Features: {len(binary_features)} (alle mit 0/1-Kodierung)")

# Robustere Berechnung des Durchschnittsumsatzes
try:
    # Methode 1: Konvertiere zu NumPy-Array und berechne den Mittelwert
    mean_umsatz = df_final['Umsatz'].to_numpy().mean()
    print(f"💰 Durchschnittsumsatz: {mean_umsatz:.2f} €")
except Exception as e:
    try:
        # Methode 2: Verwende NumPy direkt
        mean_umsatz = np.nanmean(df_final['Umsatz'].values)
        print(f"💰 Durchschnittsumsatz: {mean_umsatz:.2f} €")
    except Exception as e:
        # Fallback: Zeige den Wert ohne Formatierung
        print(f"💰 Durchschnittsumsatz: {df_final['Umsatz'].mean()} €")

print(f"\n✅ ÄNDERUNGEN DURCHGEFÜHRT:")
print(f"   • Sonntag-Feature (ist_sonntag) entfernt")
print(f"   • Warengruppen-Features mit expliziter 0/1-Binärkodierung")
print(f"   • Jahreszeiten-Features mit expliziter 0/1-Binärkodierung")
print(f"   • Wochentag-Features als Alternative verfügbar")
print(f"   • Kieler Woche und Preisindex Features hinzugefügt")

💾 Speichere finalen Trainingsdatensatz...
✅ Dataset erfolgreich gespeichert: /workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset.csv
❌ Fehler beim Speichern: No module named 'openpyxl'

🎉 TRAININGSDATENSATZ ERFOLGREICH ERSTELLT!
📁 Hauptdatei: /workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset.csv
📋 9334 Zeilen mit 34 Features
🎯 Bereit für Regression und Neuronale Netze!

📊 FINALE ZUSAMMENFASSUNG:
🥐 Warengruppen: 6
📅 Tage: 1819
🌡️ Temperaturbereich: -8.5°C - 31.4°C
🎄 Feiertage: 201 Tage
⛵ Kieler Woche: 223 Tage
🔢 Binäre Features: 20 (alle mit 0/1-Kodierung)
💰 Durchschnittsumsatz: 206.75 €

✅ ÄNDERUNGEN DURCHGEFÜHRT:
   • Sonntag-Feature (ist_sonntag) entfernt
   • Warengruppen-Features mit expliziter 0/1-Binärkodierung
   • Jahreszeiten-Features mit expliziter 0/1-Binärkodierung
   • Wochentag-Features als Alternative verfügbar
   • Kieler Woche und Preisindex Features hinzugefügt
